In [ ]:
%load_ext dotenv
%dotenv

# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [ ]:
import pandas as pd
import mappings as mappings
import schemas as schemas
import datetime
import time
import glob
import os

# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

start_time = time.time()
current_year = 2022
accounts_return_period_start_date = datetime.date(current_year - 1, 9, 10)
academy_year_start_date = datetime.date(current_year - 1, 9, 1)
academy_year_end_date = datetime.date(current_year, 8, 30)
maintained_schools_year_start_date = datetime.date(current_year, 4, 1)
maintained_schools_year_end_date = datetime.date(current_year, 3, 31)

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [ ]:
cdc = pd.read_csv('data/standing_data_cdc.csv', encoding='utf8', index_col=schemas.cdc_index_col, usecols=schemas.cdc.keys(), dtype=schemas.cdc)

cdc['Total Internal Floor Area'] = cdc.groupby(by=['URN'])['GIFA'].sum()
cdc['Proportion Area'] = (cdc['GIFA'] / cdc['Total Internal Floor Area'])
cdc['Indicative Age'] = cdc['Block Age'].fillna('').map(mappings.map_block_age).astype('Int64')
cdc['Age Score'] = cdc['Proportion Area'] * (current_year - cdc['Indicative Age'])
cdc['Age Average Score'] = cdc.groupby(by=['URN'])['Age Score'].sum()
cdc = cdc[['Total Internal Floor Area', 'Age Average Score']].drop_duplicates()

In [ ]:
cdc.to_csv('output/pre-processing/cdc.csv')
cdc

## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [ ]:
school_workforce_census = pd.read_excel('data/School_Tables_School_Workforce_Census_2022.xlsx', header=5, index_col=schemas.workforce_census_index_col, usecols=schemas.workforce_census.keys(), dtype=schemas.workforce_census, na_values=["x","u","c"], keep_default_na=True).drop_duplicates()

school_pupil_census = pd.read_csv('data/standing_data_census_pupils.csv', encoding='utf8', index_col=schemas.pupil_census_index_col, usecols=schemas.pupil_census.keys(), dtype=schemas.pupil_census).drop_duplicates()

census = school_pupil_census.join(school_workforce_census, on='URN', how='inner', rsuffix='_pupil', lsuffix='_workforce')

census.drop(labels=['full time pupils', 'headcount of pupils'], axis=1, inplace=True)
         

In [ ]:
<<<<<<< LOCAL CELL DELETED >>>>>>>
# Rename Columns
census.rename(columns={
    "Total Number of Non-Classroom-based School Support Staff, (Other school support staff plus Administrative staff plus Technicians and excluding Auxiliary staff (Full-Time Equivalent)": "FullTimeOther",
    "Total Number of Non Classroom-based School Support Staff, Excluding Auxiliary Staff (Headcount)": "FullTimeOtherHeadCount",
}, inplace=True)

In [ ]:
census.to_csv('output/pre-processing/census.csv')
census

## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [ ]:
sen = pd.read_csv('data/SEN.csv', encoding='cp1252', index_col=schemas.sen_index_col, dtype=schemas.sen, usecols=schemas.sen.keys())
sen['Percentage SEN'] = (sen['EHC plan'] / sen['Total pupils']) * 100.0
sen['Primary Need SPLD'] = sen['EHC_Primary_need_spld'] + sen['SUP_Primary_need_spld']
sen['Primary Need MLD'] = sen['EHC_Primary_need_mld'] + sen['SUP_Primary_need_mld']
sen['Primary Need SLD'] = sen['EHC_Primary_need_sld'] + sen['SUP_Primary_need_sld']
sen['Primary Need PMLD'] = sen['EHC_Primary_need_pmld'] + sen['SUP_Primary_need_pmld']
sen['Primary Need SEMH'] = sen['EHC_Primary_need_semh'] + sen['SUP_Primary_need_semh']
sen['Primary Need SLCN'] = sen['EHC_Primary_need_slcn'] + sen['SUP_Primary_need_slcn']
sen['Primary Need HI'] = sen['EHC_Primary_need_hi'] + sen['SUP_Primary_need_hi']
sen['Primary Need VI'] = sen['EHC_Primary_need_vi'] + sen['SUP_Primary_need_vi']
sen['Primary Need MSI'] = sen['EHC_Primary_need_msi'] + sen['SUP_Primary_need_msi']
sen['Primary Need PD'] = sen['EHC_Primary_need_pd'] + sen['SUP_Primary_need_pd']
sen['Primary Need ASD'] = sen['EHC_Primary_need_asd'] + sen['SUP_Primary_need_asd']
sen['Primary Need OTH'] = sen['EHC_Primary_need_oth'] + sen['SUP_Primary_need_oth']
sen.rename(columns={'prov_slcn': 'Prov_SLCN', 'prov_hi':'Prov_HI', 'prov_vi':'Prov_VI', 'prov_msi': 'Prov_MSI', 'prov_pd':'Prov_PD', 'prov_asd':'Prov_ASD', 'prov_oth':'Prov_OTH'}, inplace=True)

sen = sen[['Total pupils', 'EHC plan', 'Percentage SEN', 'Primary Need SPLD', 'Primary Need MLD', 'Primary Need SLD', 'Primary Need PMLD', 'Primary Need SEMH', 'Primary Need SLCN', 'Primary Need HI', 'Primary Need VI', 'Primary Need MSI', 'Primary Need PD', 'Primary Need ASD', 'Primary Need OTH', 'Prov_SPLD', 'Prov_MLD', 'Prov_SLD', 'Prov_PMLD', 'Prov_SEMH', 'Prov_SLCN','Prov_HI','Prov_VI','Prov_MSI','Prov_PD','Prov_ASD','Prov_OTH']] 

In [ ]:
sen.to_csv("output/pre-processing/sen.csv")
sen

## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [ ]:
ar_cell_mapping = pd.read_csv('data/AR_cell_mapping.csv', encoding='utf8', index_col=schemas.ar_cell_mapping_index_col, usecols=schemas.ar_cell_mapping.keys(), dtype=schemas.ar_cell_mapping)

ar_raw = pd.read_csv('data/AR_raw.csv', encoding='utf8', index_col=schemas.ar_index_col, usecols=schemas.ar.keys(), dtype=schemas.ar)

ar = ar_raw.reset_index().merge(ar_cell_mapping, right_on='cell', left_on='aruniquereference').set_index(schemas.ar_index_col)

pfi_schools = ar[ar['aruniquereference'] == 'BAE310-T'][['value']].map(mappings.map_is_pfi_school).rename(columns={'value': 'PFI School'})

academy_financial_position = ar[ar['aruniquereference'] == 'BAB030-T'][['value']].rename(columns={'value': 'Academy Balance'})
central_services_financial_position = ar[(ar['aruniquereference'] == 'BAB030-T') | (ar['aruniquereference'] == 'BTB030')].groupby('trustupin')['value'].sum().rename('Central Services Balance')
trust_financial_position = ar[(ar['aruniquereference'] == 'BTB030')][['trustupin', 'value']].rename(columns={'value': 'Trust Balance'}).set_index('trustupin')

teachingstaff = ar[ar['aruniquereference'] == 'BAE010-T'][['value']].rename(columns={'value': 'TeachingStaff'})
cateringsupplies = ar[ar['aruniquereference'] == 'BAE250-T'][['value']].rename(columns={'value': 'CateringSupplies'}) 


ar = (ar.join(pfi_schools, how='left')
      .join(academy_financial_position, how='left')
      .join(teachingstaff, how='left')
      .join(cateringsupplies, how='left')
      .join(trust_financial_position, on='trustupin', how='left')
      .join(central_services_financial_position, on='trustupin', how='left'))

In [ ]:
ar.to_csv('output/pre-processing/ar.csv')
ar

Create a summary table for the AR stance of each distinct academy in the table.

In [ ]:
academy_ar = ar.reset_index().drop_duplicates(subset=['academyupin'], ignore_index=True)[
    ['academyupin', 'Academy Balance', 'Trust Balance', 'Central Services Balance', 'PFI School', 'TeachingStaff', 'CateringSupplies']
].set_index('academyupin')

academy_ar['Central Services Financial Position'] = academy_ar['Central Services Balance'].map(mappings.map_is_surplus_deficit)
academy_ar['Academy Financial Position'] = academy_ar['Academy Balance'].map(mappings.map_is_surplus_deficit) 
academy_ar['Trust Financial Position'] = academy_ar['Trust Balance'].map(mappings.map_is_surplus_deficit) 

In [ ]:
academy_ar

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [ ]:
gias = pd.read_csv('data/edubasealldata20240312.csv', encoding='cp1252', 
                   index_col=schemas.gias_index_col, usecols=schemas.gias.keys(), dtype=schemas.gias)

gias_links = pd.read_csv('data/links_edubasealldata20240312.csv', encoding='cp1252', 
                         index_col=schemas.gias_links_index_col, usecols=schemas.gias_links.keys(), dtype=schemas.gias_links)

# GIAS transformations
gias['LA Establishment Number'] = gias['LA (code)'] + '-' + gias['EstablishmentNumber'].astype('string')
gias['LA Establishment Number'] = gias['LA Establishment Number'].astype('string')

gias['OpenDate'] = pd.to_datetime(gias['OpenDate'], dayfirst=True, format='mixed')
gias['CloseDate'] = pd.to_datetime(gias['CloseDate'], dayfirst=True, format='mixed')
gias['SchoolWebsite'] = gias['SchoolWebsite'].fillna('').map(mappings.map_school_website)
gias['Boarders (name)'] = gias['Boarders (name)'].fillna('').map(mappings.map_boarders)
gias['OfstedRating (name)'] = gias['OfstedRating (name)'].fillna('').map(mappings.map_ofsted_rating)
gias['NurseryProvision (name)'] = gias['NurseryProvision (name)'].fillna('')
gias['OfficialSixthForm (name)'] = gias['OfficialSixthForm (name)'].fillna('').map(mappings.map_sixth_form)
gias['AdmissionsPolicy (name)'] = gias['AdmissionsPolicy (name)'].fillna('').map(mappings.map_admission_policy)
gias['HeadName'] = gias['HeadTitle (name)'] + ' ' + gias['HeadFirstName'] + ' ' + gias['HeadLastName']
gias['IsFederation'] = gias['Federation']
gias['IsPartOfFederation']

In the following cell, we find all the predecessor and merged links. The links are then Ranked by URN and order by 'Link Established Date'. The linked GAIS data in then joined to the base GIAS data. This creates the overall school data set. This dataset is then filtered for schools that are open (CloseDate is null) and the schools with nested links that are Ranked 1.

In [ ]:
gias_links = gias_links[
    gias_links['LinkType'].isin(['Predecessor', 
                                 'Predecessor - amalgamated', 
                                 'Predecessor - Split School', 
                                 'Predecessor - merged', 
                                 'Merged - expansion of school capacity', 
                                 'Merged - change in age range'])
].sort_values(by='LinkEstablishedDate', ascending=False)

gias_links['Rank'] = gias_links.groupby('URN').cumcount() + 1
gias_links['Rank'] = gias_links['Rank'].astype('Int64')

schools = gias.join(gias_links, on='URN', how='left', rsuffix='_links', lsuffix='_school').sort_values(by='URN')

schools = schools[
    schools['CloseDate'].isna() & ((schools['Rank'] == 1) | (schools['Rank'].isna()))
].drop(columns=['LinkURN', 'LinkName', 'LinkType', 'LinkEstablishedDate', 'Rank'])

In [ ]:
schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [ ]:
academies_list = pd.read_csv('data/master_list_raw.csv', encoding='utf8', index_col=schemas.academy_master_list_index_col, dtype=schemas.academy_master_list, usecols=schemas.academy_master_list.keys()).rename(columns={'UKPRN': 'Academy UKPRN'})

academies_base = academies_list.merge(schools.reset_index(), left_index=True, right_on='LA Establishment Number').set_index('URN')

academies = (academies_base.merge(census, on='URN', how='left')
             .merge(sen, on='URN', how='left')
             .merge(cdc, on='URN', how='left')
             .merge(academy_ar, left_on='Academy UPIN', right_index=True, how='left'))

academies['Type of Provision - Phase'] = academies.apply(lambda df: mappings.map_academy_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

# Bizarre I shouldn't need this as this is coming from the original GIAS dataset, but I seem to have to do this twice. 
academies['NurseryProvision (name)'] = academies['NurseryProvision (name)'].fillna('')
academies['NurseryProvision (name)'] = academies.apply(lambda df: mappings.map_nursery(df['NurseryProvision (name)'], df['Type of Provision - Phase']), axis=1)

academies['Status'] = academies.apply(lambda df: mappings.map_academy_status(pd.to_datetime(df['Date left or closed if in period']), 
                                                                             pd.to_datetime(df['Valid to']), 
                                                                             pd.to_datetime(df['OpenDate']), 
                                                                             pd.to_datetime(df['CloseDate']), 
                                                                             pd.to_datetime(accounts_return_period_start_date), pd.to_datetime(academy_year_start_date), pd.to_datetime(academy_year_end_date)), axis=1)

academies['SchoolPhaseType'] = academies.apply(lambda df: mappings.map_school_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

academies.rename(columns={
    'UKPRN_x':'UKPRN',
    'Number of Pupils': 'Number of pupils',
    '% of pupils known to be eligible for free school meals (Performa': 'Percentage Free school meals'
}, inplace=True)

In [ ]:
academies.to_csv('output/pre-processing/academies.csv')
academies.sort_index()

Merge required census and cdc data to the maintained schools data set

In [ ]:
# Load raw list from CSV
maintained_schools_list = pd.read_csv('data/maintained_schools_raw.csv', encoding='utf8', index_col=schemas.maintained_schools_master_list_index_col, usecols=schemas.maintained_schools_master_list.keys(), dtype=schemas.maintained_schools_master_list)

In [ ]:
# Merge maintained_schools_base with schools (metadata from GIAS) & rename PFI column
maintained_schools = maintained_schools_list.merge(schools.reset_index(), left_index=True, right_on='URN')


In [ ]:
# Merge in census and cdc data
maintained_schools = (maintained_schools
                      .merge(sen, on='URN', how='left')
                      .merge(census, on='URN', how='left')
                      .merge(cdc, on='URN', how='left'))



In [ ]:
# Compute columns
maintained_schools['PFI'] = maintained_schools['PFI'].map(lambda x: 'PFI school' if x == 'Y' else 'Non-PFI school')
maintained_schools['Status'] = maintained_schools.apply(lambda df: mappings.map_maintained_school_status(df['OpenDate'], df['CloseDate'], df['Period covered by return (months)'], pd.to_datetime(maintained_schools_year_start_date), pd.to_datetime(maintained_schools_year_end_date)), axis=1)
maintained_schools['School Balance'] = maintained_schools['Total Income   I01 to I18'] - maintained_schools['Total Expenditure  E01 to E32']
maintained_schools['School Financial Position'] = maintained_schools['School Balance'].map(mappings.map_is_surplus_deficit)
maintained_schools['SchoolPhaseType'] = maintained_schools.apply(lambda df: mappings.map_school_phase_type(df['TypeOfEstablishment (code)'], df['Overall Phase']), axis=1)
maintained_schools['Partial Years Present'] = maintained_schools['Period covered by return (months)'].map(lambda x: x != 12)
maintained_schools['Did Not Submit'] = maintained_schools['Did Not Supply flag'].map(lambda x: x == 1)

In [ ]:
# Rename columns
maintained_schools.rename(columns={
    'PFI': 'PFI School',
    '% of pupils eligible for FSM': 'Percentage Free school meals',
    'No Pupils': 'Number of pupils',
    'I07  Other grants and payments': 'Other grants and payments'
}, inplace=True)

In [ ]:
maintained_schools.set_index('URN', inplace=True)

In [ ]:
maintained_schools

In [ ]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv')


## Federation Capture




In [ ]:
group_links = pd.read_csv('data/alllinksdata20240417.csv', encoding='unicode-escape',  index_col=schemas.groups_index_col, usecols=schemas.groups.keys(), dtype=schemas.groups)

In [ ]:
# filter lead schools out of the maintained schools list
federations = maintained_schools[['LAEstab']][maintained_schools['Federation']=='Lead school'].copy()
# join 
federations = federations.join(group_links[['Group Name','Group UID','Closed Date']])


# remove federations with an associated closed date
federations = federations.loc[federations['Closed Date'].isna()]

# federations with a UID listed in the GIAS groups data are referred to as "Hard" federations
# while federations not listed in GIAS are referred to as "Soft" federations.
# Soft federation UIDs are a combination of their URN and LAEstab codes.

# create mask for soft federations
mask = federations['Group UID'].isna()

hard_federations = federations.loc[~mask].copy()
soft_federations = federations.loc[mask].copy()

# define members list for hard federations
group_links['Members'] = group_links.index
hard_members = group_links[['Members','Group UID']].groupby('Group UID').agg(list)

hard_federations = hard_federations.join(hard_members, on='Group UID')

# Rename columns
hard_federations.rename(columns={
    'Group Name': 'FederationName',
    'Group UID': 'FederationUid',
}, inplace=True)

# for the soft federations
soft_federations['FederationUid'] = soft_federations.index.astype(str) + soft_federations['LAEstab'].astype(str)

# Rename columns
soft_federations.rename(columns={
    'Group Name': 'FederationName',
    'Group UID': 'FederationUid',
}, inplace=True)

# TODO - add in soft federation members and names (currently no mapping available)

In [ ]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

In [ ]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')